### Импортируем библиотеки

In [5]:
import numpy as np
import pandas as pd
from docx import Document

### Считываю исходную таблицу из файла

In [2]:
def parse_docx_table(filename: str):
    document = Document(filename)

    tables = []
    for table in document.tables:
        # список списков с пустыми строками - структура будущего датафрейма
        df = [['' for x in range(len(table.columns))] for y in range(len(table.rows))]
        # заполняем дф данными из табл
        for x, row in enumerate(table.rows):
            for y, cell in enumerate(row.cells):
                if cell.text:
                    df[x][y] = cell.text
        tables.append(pd.DataFrame(df))

    return tables

In [3]:
df = parse_docx_table('DE_task_table.docx')[0]

NameError: name 'Document' is not defined

In [5]:
df = tables[0]
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

In [6]:
df

,tab_num,start_date,finish_date,wday_type01,wday_type02,wday_type03,wday_type04,wday_type05,wplace_type,end_da
1,15123,02.09.2020,31.12.9999,0,0,0,0,0,0,31.10.2020
2,16234,20.09.2020,30.10.2020,0,0,1,1,0,2,
3,17345,28.09.2020,25.10.2020,1,0,0,0,0,2,
4,17345,26.10.2020,31.12.2020,1,1,1,1,1,1,
5,18456,02.09.2020,31.12.9999,2,2,2,2,2,3,30.09.2020
6,19567,02.09.2020,31.12.2020,3,3,3,3,3,4,


In [7]:
df['start_date']= pd.to_datetime(df['start_date'], format="%d.%m.%Y", errors = 'coerce')
df['finish_date']= pd.to_datetime(df['finish_date'], format="%d.%m.%Y", errors = 'coerce')

/tmp/ipykernel_32671/587058181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['start_date']= pd.to_datetime(df['start_date'], format="%d.%m.%Y", errors = 'coerce')
/tmp/ipykernel_32671/587058181.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['finish_date']= pd.to_datetime(df['finish_date'], format="%d.%m.%Y", errors = 'coerce')


In [8]:
df

,tab_num,start_date,finish_date,wday_type01,wday_type02,wday_type03,wday_type04,wday_type05,wplace_type,end_da
1,15123,2020-09-02,NaT,0,0,0,0,0,0,31.10.2020
2,16234,2020-09-20,2020-10-30,0,0,1,1,0,2,
3,17345,2020-09-28,2020-10-25,1,0,0,0,0,2,
4,17345,2020-10-26,2020-12-31,1,1,1,1,1,1,
5,18456,2020-09-02,NaT,2,2,2,2,2,3,30.09.2020
6,19567,2020-09-02,2020-12-31,3,3,3,3,3,4,


In [9]:
df.loc[df['finish_date'].isnull(), 'finish_date'] = df['end_da']

/home/varsey-fabuza/PycharmProjects/ML-magic/dashboard-formation/env/lib/python3.10/site-packages/pandas/core/arrays/datetimes.py:339: UserWarning: Parsing '31.10.2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return cls._from_sequence_not_strict(scalars, dtype=dtype, copy=copy)
/home/varsey-fabuza/PycharmProjects/ML-magic/dashboard-formation/env/lib/python3.10/site-packages/pandas/core/arrays/datetimes.py:339: UserWarning: Parsing '30.09.2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return cls._from_sequence_not_strict(scalars, dtype=dtype, copy=copy)


In [10]:
df

,tab_num,start_date,finish_date,wday_type01,wday_type02,wday_type03,wday_type04,wday_type05,wplace_type,end_da
1,15123,2020-09-02,2020-10-31,0,0,0,0,0,0,31.10.2020
2,16234,2020-09-20,2020-10-30,0,0,1,1,0,2,
3,17345,2020-09-28,2020-10-25,1,0,0,0,0,2,
4,17345,2020-10-26,2020-12-31,1,1,1,1,1,1,
5,18456,2020-09-02,2020-09-30,2,2,2,2,2,3,30.09.2020
6,19567,2020-09-02,2020-12-31,3,3,3,3,3,4,


In [11]:
dict(zip(range(df.shape[0]), df.start_date.to_list()))

{0: Timestamp('2020-09-02 00:00:00'),
 1: Timestamp('2020-09-20 00:00:00'),
 2: Timestamp('2020-09-28 00:00:00'),
 3: Timestamp('2020-10-26 00:00:00'),
 4: Timestamp('2020-09-02 00:00:00'),
 5: Timestamp('2020-09-02 00:00:00')}

In [12]:
start_date_dict = dict(zip(range(df.shape[0]), df.start_date.to_list()))

In [13]:
finish_date_dict = dict(zip(range(df.shape[0]), df.finish_date.to_list()))

In [14]:
dash = pd.DataFrame()
for num, tab_num in enumerate(df.tab_num.to_list()):
    interim = pd.DataFrame()
    interim['ymd_date'] = pd.date_range(start='2020-09-01', end='2020-12-31', freq='D')
    interim['weekday'] = interim['ymd_date'].dt.dayofweek
    interim['week'] = interim['ymd_date'].dt.isocalendar().week
    interim['week'] = interim['week'] - interim['week'].min()
    interim['halfweek'] = interim['week'] // 2
    interim['tab_num'] = tab_num
    interim['start_date'] = start_date_dict[num]
    interim['finish_date'] = finish_date_dict[num]
    dash = pd.concat([dash, interim])
dash['to_be_at_office'] = np.nan

In [15]:
dash_merged = pd.concat(
    [
        dash[~dash.tab_num.isin(['17345'])],
        pd.merge(
            dash.loc[(dash['tab_num'] == '17345')][['ymd_date', 'weekday', 'tab_num']],
            dash.loc[
                (dash['tab_num'] == '17345') &
                (dash['ymd_date'] <= dash['finish_date']) &
                (dash['ymd_date'] >= dash['start_date'])
            ],
            how='left'
        ).drop_duplicates()
    ]
)

In [16]:
732 - dash[dash['tab_num'] == '17345'].shape[0] + 122

610

In [17]:
dash.shape, dash_merged.shape, 122*5

((732, 8), (610, 8), 610)

In [18]:
dash.sample(5)

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
89,2020-11-29,6,12,6,17345,2020-09-28,2020-10-25,NaN
59,2020-10-30,4,8,4,17345,2020-09-28,2020-10-25,NaN
13,2020-09-14,0,2,1,16234,2020-09-20,2020-10-30,NaN
93,2020-12-03,3,13,6,15123,2020-09-02,2020-10-31,NaN
82,2020-11-22,6,11,5,19567,2020-09-02,2020-12-31,NaN


In [19]:
dash_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 610 entries, 0 to 121
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ymd_date         610 non-null    datetime64[ns]
 1   weekday          610 non-null    int64         
 2   week             583 non-null    UInt32        
 3   halfweek         583 non-null    UInt32        
 4   tab_num          610 non-null    object        
 5   start_date       583 non-null    datetime64[ns]
 6   finish_date      583 non-null    datetime64[ns]
 7   to_be_at_office  0 non-null      float64       
dtypes: UInt32(2), datetime64[ns](3), float64(1), int64(1), object(1)
memory usage: 39.3+ KB


In [20]:
df.tab_num.unique()[0]

'15123'

In [21]:
print(df.tab_num.to_list()[0])
tab_num_index = 0
dash_merged.loc[
    (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
    (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['weekday'] != 5) &
    (dash_merged['weekday'] != 6),
    'to_be_at_office'
] = 1

15123


In [22]:
dash_merged[dash_merged['tab_num'] == '15123']

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,15123,2020-09-02,2020-10-31,NaN
1,2020-09-02,2,0,0,15123,2020-09-02,2020-10-31,1.0
2,2020-09-03,3,0,0,15123,2020-09-02,2020-10-31,1.0
3,2020-09-04,4,0,0,15123,2020-09-02,2020-10-31,1.0
4,2020-09-05,5,0,0,15123,2020-09-02,2020-10-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,15123,2020-09-02,2020-10-31,NaN
118,2020-12-28,0,17,8,15123,2020-09-02,2020-10-31,NaN
119,2020-12-29,1,17,8,15123,2020-09-02,2020-10-31,NaN
120,2020-12-30,2,17,8,15123,2020-09-02,2020-10-31,NaN


In [23]:
print(df.tab_num.to_list()[3])

17345


In [24]:
tab_num_index = 3
dash_merged.loc[
    (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
    (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['weekday'] != 5) &
    (dash_merged['weekday'] != 6),
    'to_be_at_office'
] = 0

In [25]:
dash_merged[dash_merged['tab_num'] == '17345']

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,<NA>,<NA>,17345,NaT,NaT,NaN
1,2020-09-02,2,<NA>,<NA>,17345,NaT,NaT,NaN
2,2020-09-03,3,<NA>,<NA>,17345,NaT,NaT,NaN
3,2020-09-04,4,<NA>,<NA>,17345,NaT,NaT,NaN
4,2020-09-05,5,<NA>,<NA>,17345,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,17345,2020-10-26,2020-12-31,NaN
118,2020-12-28,0,17,8,17345,2020-10-26,2020-12-31,0.0
119,2020-12-29,1,17,8,17345,2020-10-26,2020-12-31,0.0
120,2020-12-30,2,17,8,17345,2020-10-26,2020-12-31,0.0


In [26]:
tab_num_index = 2
dash_merged.loc[
    (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
    (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
    (dash_merged['weekday'] != 5) &
    (dash_merged['weekday'] != 6) &
    (dash_merged['weekday'] == 0),
    'to_be_at_office'
] = 0

In [27]:
tab_num_index = 2
dash_merged.loc[
    (
        (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] < finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] > start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    ) &
    (
        (dash_merged['weekday'].isin([1, 2, 3, 4]))
    ),
    'to_be_at_office'
] = 1

In [28]:
tab_num_index = 1
dash_merged.loc[
    (
        (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    ) &
    (
        (dash_merged['weekday'].isin([0, 1, 4]))
    ),
    'to_be_at_office'
] = 1

In [29]:
tab_num_index = 1
dash_merged.loc[
    (
        (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([2, 3]))
    ),
    'to_be_at_office'
] = 0

In [30]:
dash_merged[dash_merged['tab_num'] == df.tab_num.to_list()[1]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,16234,2020-09-20,2020-10-30,NaN
1,2020-09-02,2,0,0,16234,2020-09-20,2020-10-30,NaN
2,2020-09-03,3,0,0,16234,2020-09-20,2020-10-30,NaN
3,2020-09-04,4,0,0,16234,2020-09-20,2020-10-30,NaN
4,2020-09-05,5,0,0,16234,2020-09-20,2020-10-30,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,16234,2020-09-20,2020-10-30,NaN
118,2020-12-28,0,17,8,16234,2020-09-20,2020-10-30,NaN
119,2020-12-29,1,17,8,16234,2020-09-20,2020-10-30,NaN
120,2020-12-30,2,17,8,16234,2020-09-20,2020-10-30,NaN


In [31]:
tab_num_index = 4
dash_merged.loc[
    (
        (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['week'] % 2 == 0)
    ),
    'to_be_at_office'
] = 0

In [32]:
tab_num_index = 4
dash_merged.loc[
    (
        (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['week'] % 2 == 1)
    ),
    'to_be_at_office'
] = 1

In [33]:
dash_merged[dash_merged['tab_num'] == df.tab_num.to_list()[4]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,18456,2020-09-02,2020-09-30,NaN
1,2020-09-02,2,0,0,18456,2020-09-02,2020-09-30,0.0
2,2020-09-03,3,0,0,18456,2020-09-02,2020-09-30,0.0
3,2020-09-04,4,0,0,18456,2020-09-02,2020-09-30,0.0
4,2020-09-05,5,0,0,18456,2020-09-02,2020-09-30,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,18456,2020-09-02,2020-09-30,NaN
118,2020-12-28,0,17,8,18456,2020-09-02,2020-09-30,NaN
119,2020-12-29,1,17,8,18456,2020-09-02,2020-09-30,NaN
120,2020-12-30,2,17,8,18456,2020-09-02,2020-09-30,NaN


In [34]:
tab_num_index = 5
dash_merged.loc[
    (
        (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['halfweek'] % 2 == 0)
    ),
    'to_be_at_office'
] = 0

In [35]:
tab_num_index = 5
dash_merged.loc[
    (
        (dash_merged['tab_num'] == df.tab_num.to_list()[tab_num_index]) &
        (dash_merged['ymd_date'] <= finish_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['ymd_date'] >= start_date_dict[tab_num_index].to_datetime64()) &
        (dash_merged['weekday'] != 5) &
        (dash_merged['weekday'] != 6)
    )
        &
    (
        (dash_merged['weekday'].isin([0, 1, 2, 3, 4]))
    )
            &
    (
        (dash_merged['halfweek'] % 2 == 1)
    ),
    'to_be_at_office'
] = 1

In [36]:
dash_merged[dash_merged['tab_num'] == df.tab_num.to_list()[5]]

,ymd_date,weekday,week,halfweek,tab_num,start_date,finish_date,to_be_at_office
0,2020-09-01,1,0,0,19567,2020-09-02,2020-12-31,NaN
1,2020-09-02,2,0,0,19567,2020-09-02,2020-12-31,0.0
2,2020-09-03,3,0,0,19567,2020-09-02,2020-12-31,0.0
3,2020-09-04,4,0,0,19567,2020-09-02,2020-12-31,0.0
4,2020-09-05,5,0,0,19567,2020-09-02,2020-12-31,NaN
...,...,...,...,...,...,...,...,...
117,2020-12-27,6,16,8,19567,2020-09-02,2020-12-31,NaN
118,2020-12-28,0,17,8,19567,2020-09-02,2020-12-31,0.0
119,2020-12-29,1,17,8,19567,2020-09-02,2020-12-31,0.0
120,2020-12-30,2,17,8,19567,2020-09-02,2020-12-31,0.0


In [37]:
# TO-DO поменять формат даты

In [39]:
dash_merged[['tab_num', 'ymd_date', 'to_be_at_office']].sort_values(by=['tab_num', 'ymd_date']).reset_index(drop=True)

,tab_num,ymd_date,to_be_at_office
0,15123,2020-09-01,NaN
1,15123,2020-09-02,1.0
2,15123,2020-09-03,1.0
3,15123,2020-09-04,1.0
4,15123,2020-09-05,NaN
...,...,...,...
605,19567,2020-12-27,NaN
606,19567,2020-12-28,0.0
607,19567,2020-12-29,0.0
608,19567,2020-12-30,0.0
